# Projeto: Generative

**Autores:** [Lucas Lima](https://github.com/lucasouzamil), [Henrique Badin](https://github.com/henriquefbadin) e [Eduardo Selber](https://github.com/eduselber).

**Resumo:** Este notebook apresenta o relatório do projeto que explora um pipeline de geração de imagens usando **Stable Diffusion** (sugerido via ComfyUI). O foco está em explicar a arquitetura, apresentar experimentos (5 exemplos input→output), discutir resultados e cumprir os critérios da avaliação.

## 1. Introdução
Este projeto investiga técnicas de geração de imagens condicionadas por texto usando Stable Diffusion. O objetivo principal é construir e documentar um pipeline (Text → Image), explicar a arquitetura empregada, fornecer exemplos de entrada e saída e discutir os resultados de forma clara e objetiva, seguindo a rubrica do trabalho.


## 2. Arquitetura do Pipeline de Geração

O pipeline completo é composto por duas etapas principais: a geração da imagem 2D e a reconstrução tridimensional dessa imagem. A seguir, descrevem-se os componentes envolvidos em cada fase.

### 2.1 Arquitetura do Pipeline de Geração 2d

O pipeline é composto pelos seguintes componentes:

1. **Load Checkpoint**  
   Carrega o modelo base, o CLIP e o VAE utilizados na geração.

2. **Load LoRAs**  
   Aplica sequencialmente as LoRAs ao modelo principal, ajustando tanto o modelo como o CLIP.

3. **CLIP Text Encode (Prompt Positivo e Negativo)**  
   Converte os textos dos prompts em embeddings que guiam o modelo durante a geração.

4. **Empty Latent Image**  
   Cria o espaço latente inicial onde a imagem será sintetizada.

5. **KSampler**  
   Efetua o processo de difusão, refinando o latent até formar o conceito requisitado.

6. **VAE Decode**  
   Converte o latent final em uma imagem RGB.

7. **Save Image**  
   Salva a imagem final no disco.


### 2.2. Pipeline de Reconstrução 3D

Após a geração da imagem 2D, o pipeline segue para a etapa de reconstrução tridimensional utilizando o modelo Hunyuan3D-V2.

8. **Image-Only Checkpoint Loader (Hunyuan3D)**  
Carrega o modelo especializado em reconstrução 3D, incluindo o encoder visual e o VAE próprio desse módulo.

9. **CLIP Vision Encode**
Extrai embeddings visuais da imagem 2D gerada, capturando seus atributos de forma robusta para orientar o processo 3D.

10. **Hunyuan3Dv2 Conditioning**  
Converte os embeddings visuais em condicionamentos positivos e negativos adequados para a difusão volumétrica.

11. **Empty Latent Hunyuan3Dv2**  
Cria o espaço latente tridimensional inicial, representado em voxels, que servirá como estrutura base para a reconstrução.

12. **KSampler (3D)**  
Executa o processo de difusão 3D, refinando progressivamente o volume latente até formar a geometria correspondente à imagem 2D.

13. **VAE Decode Hunyuan3D**  
Decodifica o latent volumétrico, produzindo uma representação voxelizada do modelo tridimensional.

14. **VoxelToMesh**  
Converte o volume voxelizado em uma malha poligonal utilizando algoritmos de reconstrução de superfície.

15. **SaveGLB**  
Exporta a malha final no formato GLB, possibilitando visualização, manipulação e uso do modelo 3D em softwares externos.

### Códio de implementação

```python
import subprocess
import os
from os import path
import warnings
from IPython.display import clear_output
warnings.filterwarnings('ignore')
from multiprocessing import Process
import sys
import time

BASE = os.getcwd()

def download_from_civitai(URL_CITVITAI, PASTA_ONDE_DEVE_SER_INSTALADA, NOME_DO_ARQUIVO):
    tok = COLE_SEU_TOKEB_CTIVITAI_AQUI
    model_url = f"{url}&token={tok}"
    save_path = f"{BASE}/ComfyUI/models/{folder}/{name}"
    get_ipython().system(f'wget -O "{save_path}" "{model_url}"')


def delete_model(PASTA, NOME):
    %cd $BASE/ComfyUI/models/$folder
    all_names = os.listdir()
    print("O caminho atual é: ", os.getcwd())
    print("Todos os arquivos no diretório atual: ", all_names)
    
    if name in all_names:
        %rm $name
        print("O ARQUIVO DO MODELO FOI EXCLUÍDO COM SUCESSO")
    else:
        print("O ARQUIVO DO MODELO FORNECIDO NÃO FOI ENCONTRADO")

%cd $BASE
!git clone https://github.com/comfyanonymous/ComfyUI.git
%cd ComfyUI
get_ipython().system('git pull')
!pip install - torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu124
!pip install -r requirements.txt
!pip install --upgrade gguf
!pip install bitsandbytes>=0.43.0
!mamba install openssh -y
 
%cd $BASE/ComfyUI/custom_nodes
!git clone https://github.com/ltdrdata/ComfyUI-Manager.git
%cd ComfyUI-Manager
get_ipython().system('git pull')
!pip install -r requirements.txt

%cd $BASE/ComfyUI/custom_nodes
!git clone https://github.com/chrisgoringe/cg-use-everywhere.git  
!git clone https://github.com/pythongosssss/ComfyUI-Custom-Scripts.git  
!git clone https://github.com/WASasquatch/was-node-suite-comfyui.git  
!git clone https://github.com/rgthree/rgthree-comfy.git  
!git clone https://github.com/city96/ComfyUI-GGUF  
!git clone https://github.com/crystian/ComfyUI-Crystools.git  
%cd ComfyUI-Crystools
!pip install -r requirements.txt
!git clone https://github.com/kijai/ComfyUI-KJNodes.git
!git clone https://github.com/11cafe/comfyui-workspace-manager.git
%cd /kaggle/working/ComfyUI/custom_nodes
!pip install -r requirements.txt


clear_output(wait=True)
%cd $BASE/ComfyUI

print(f"Current path: {BASE}/ComfyUI")
print("COMFYUI MANAGER + CUSTOM NODES FORAM INSTALADOS COM SUCESSO!👍")

### Dreamshaper_8
model_url = "https://civitai.com/api/download/models/128713?type=Model&format=SafeTensor&size=pruned&fp=fp16"
model_name = "dreamshaper_8.safetensors"
%cd /kaggle/working/ComfyUI/models/checkpoints
get_ipython().system(f'wget -O "{model_name}" "{model_url}"')

### blindbox_V1Mix
model_url = "https://civitai.com/api/download/models/32988?type=Model&format=SafeTensor&size=full&fp=fp16"
model_name = "blindbox_V1Mix.safetensors"
%cd /kaggle/working/ComfyUI/models/loras
get_ipython().system(f'wget -O "{model_name}" "{model_url}"')

### MoXinV1
model_url = "https://civitai.com/api/download/models/14856?type=Model&format=SafeTensor&size=full&fp=fp16"
model_name = "MoXinV1.safetensors"
%cd /kaggle/working/ComfyUI/models/loras
get_ipython().system(f'wget -O "{model_name}" "{model_url}"')

### hunyuan3d-dit-v2-mv
model_url = "https://huggingface.co/Comfy-Org/hunyuan3D_2.1_repackaged/resolve/main/hunyuan_3d_v2.1.safetensors"
model_name = "hunyuan_3d_v2.1.safetensors"
%cd /kaggle/working/ComfyUI/models/checkpoints
get_ipython().system(f'wget -O "{model_name}" "{model_url}"')

%cd /kaggle/working/ComfyUI
print("LoRA Template e WAN Template FORAM INSTALADOS COM SUCESSO! 👍")

%cd $BASE/ComfyUI

!touch log.txt
open('log.txt', 'w').close()
clear_output(wait=True)

def run_app():
    cmd = f"python {BASE}/ComfyUI/main.py & ssh -o StrictHostKeyChecking=no -p 80 -R0:localhost:8188 a.pinggy.io > log.txt"
    get_ipython().system(cmd)
    
def print_url():
    print("waiting for output")
    time.sleep(2)
    sys.stdout.flush()
    
    found = False
    with open('log.txt', 'r') as file:
        end_word = '.pinggy.link'
        for line in file:
            start_index = line.find("http:")
            if start_index != -1:
                end_index = line.find(end_word, start_index)
                if end_index != -1:
                    print("#######")
                    print("URL: " + line[start_index:end_index + len(end_word)])
                    print("#######")
                    found = True
    if not found:
        print_url()
    else:
        with open('log.txt', 'r') as file:
            for line in file:
                print(line)
    
p_app = Process(target=run_app)
p_url = Process(target=print_url)
p_app.start()
p_url.start()
p_app.join()
p_url.join()

```

## 4. Metodologia experimental (como os experimentos foram planejados)

- Estruturamos os testes em dois âmbitos principais de variação de parâmetros:
    1. número de samples

    2. valor de denoise do K-Sampler em cada workflow.

- Na primeira etapa, realizamos experimentos apenas no workflow com LoRA, variando samples e denoise diretamente nesse fluxo, mantendo o workflow de geração 3D inalterado.

- Na segunda etapa, mantivemos o workflow do LoRA fixo (com a melhor configuração identificada na etapa anterior) e passamos a variar os mesmos parâmetros (samples e denoise) no workflow de geração de imagem utilizado pelo Image-to-3D.

- Para ambos os workflows (LoRA e Image3D), testamos sistematicamente os seguintes valores de parâmetros:
    - Samples: 10, 30, 50
    - Denoise: 0.7, 0.85, 1.0

- Em todos os testes, mantivemos os demais parâmetros dos nós (modelo base, LoRA selecionado, prompts e resolução) constantes, para isolar o impacto específico da variação de samples e denoise na qualidade visual das imagens e na geração dos modelos 3D.

## Testes fazendo alterações no workflow do LoRA:

#### Denoise = 0.7 e Steps = 30:

<div style="display: flex; justify-content: center; gap: 20px;">
    <img src="../../imgs/view.png" width="300">
    <img src="../../imgs/view3d.png" width="300">
</div>

#### Denoise = 0.85 e Steps = 30:

<div style="display: flex; justify-content: center; gap: 20px;">
    <img src="../imgs/view2.png" width="300">
    <img src="../imgs/view23d.png" width="300">
</div>

#### Denoise = 1.0 e Steps = 30:

<div style="display: flex; justify-content: center; gap: 20px;">
    <img src="../imgs/view3.png" width="300">
    <img src="../imgs/view33d.png" width="300">
</div>


#### Denoise = 1.0 e Steps = 50 :

<div style="display: flex; justify-content: center; gap: 20px;">
    <img src="../imgs/view4.png" width="300">
    <img src="../imgs/view43d.png" width="300">
</div>

#### Denoise = 1.0 e Steps 10:

<div style="display: flex; justify-content: center; gap: 20px;">
    <img src="../imgs/view5.png" width="300">
    <img src="../imgs/view53d.png" width="300">
</div>

## Testes fazendo alterações no workflow do Image-to-3D:

#### Denoise = 0.7 e Steps = 30:

<div style="display: flex; justify-content: center; gap: 20px;">
    <img src="../imgs/view6.png" width="300">
    <img src="../imgs/view623d.png" width="300">
</div>

#### Denoise = 0.85 Steps = 30:

<div style="display: flex; justify-content: center; gap: 20px;">
    <img src="../imgs/view7.png" width="300">
    <img src="../imgs/view73d.png" width="300">
</div>

#### Denoise = 1.0 e Steps = 30:

<div style="display: flex; justify-content: center; gap: 20px;">
    <img src="../imgs/view3.png" width="300">
    <img src="../imgs/view33d.png" width="300">
</div>

####  Denoise = 1.0 e Steps = 50:

<div style="display: flex; justify-content: center; gap: 20px;">
    <img src="../imgs/view8.png" width="300">
    <img src="../imgs/view83d.png" width="300">
</div>

#### Denoise = 1.0 e Steps 10:

<div style="display: flex; justify-content: center; gap: 20px;">
    <img src="../imgs/view9.png" width="300">
    <img src="../imgs/view93d.png" width="300">
</div>

# Conclusão dos testes

## A partir das variações de parâmetros, dá para tirar alguns aprendizados bem claros:

### **Denoise 0.7 e 0.85**
Com esses valores mais baixos, as imagens perderam força. Os detalhes ficam pobres, partes como mãos e a arma aparecem deformadas e a composição como um todo parece menos fiel ao que o prompt pedia. É como se a imagem não tivesse maturado o suficiente.

### **Denoise 1.0** 
Nesse nível, o modelo realmente “fecha” a ideia. As imagens ficam mais consistentes, com estrutura mais clara e coerente, incluindo como ,por exeplo,  pose, roupa, arma, tudo se encaixa melhor. Mesmo sendo geração de IA, o resultado transmite muito mais estabilidade e proximidade de algo plausível.

### **Steps 10, 30 e 50 (com denoise 1.0)**

Com **10 steps** , a imagem parece interrompida no meio; vários trechos saem distorcidos.  
Com **30 steps** , o resultado já é sólido e utilizável.  
Com **50 steps**, há um refinamento extra: o rosto melhora, pequenos artefatos desaparecem e a imagem ganha um acabamento mais profissional.  

Em resumo, para esse conjunto de testes, a combinação denoise 1.0 com 30 a 50 steps entregou os resultados mais completos, estáveis e coerentes,  tanto para uso direto em 2D quanto pensando no pipeline para 3D depois.